## Jupyter Notebook zur Berechnung der prozentualen Übereinstimmung der Lehrer

Die Fragebögen aus den Maurer- und DWA- Befragungen wurden an Lehrkräfte im Zielgebiet verschickt. Gegeben der kurzen Zeitspanne zwischen den Untersuchungen und der Vielzahl der Fragebögen, liegt die Vermutung nahe, dass einige Lehrkräfte an beiden Befragungen teilgenommen haben.
Die erste relevante Frage hierbei ist, wie viel Prozent der Lehrkräfte überschneiden sich? Dieses Notebook soll diese Frage beantworten.

Uns liegt eine Tabelle vor, in der für jeden Befragungsort geschaut wurde, ob die Lehrkraft übereinstimmt oder nicht. Dabei ist zu beachten, dass lediglich die transkribierten Lehrernamen aus der DWA Befragung vorliegen. Die Lehrernamen der Maurerbefragung wurden bis jetzt noch nicht transkribiert. Außerdem werden Mehrfachnennungen von Lehrkräften in einem Bogen in der Tabelle auf mehrere Spalten aufgeteilt. Das führt dazu, dass die Fragebögen (sowohl DWA als auch Maurer) oft mehrfach in der Tabelle vorkommen und die Orte entsprechend auch mehrfach vorkommen.

Im Folgenden werden also mehrere, unterschiedlich komplexe Ansätze ausprobiert, um eine prozentuale Annäherung zu erreichen.

In [1]:
import pandas as pd

df = pd.read_csv("/home/kopatsch/Masterarbeit/MA/masterarbeit/data/csv_teacher/DWA_Maurer_Lehrerabgleich.csv", sep = "\t")
df["coord"] = df["LONG"].astype(str) + ", " + df["LAT"].astype(str)

In [2]:
df2 = pd.read_csv("/home/kopatsch/Masterarbeit/MA/masterarbeit/data/csv_gesamt/all_words.csv", sep = "\t")
df2["coord"] = df2["Longitude"].astype(str) + ", " + df2["Latitude"].astype(str)

## 1. Ansatz

Die Tabelle wird nicht bereinigt. Aus allen vorkommenden Daten wird ermittelt, welcher Prozentsatz mit "1.0" (Übereinstimmung) bewertet wurde. Das Ergebnis sind ca. 51% Übereinstimmung. Dies beantwortet vermutlich am ehesten die Frage, wie viel Prozent der Lehrer übereinstimmen.

In [3]:
wert1 = len(df["Lehrer.übereinstimmend_clean"])
wert = [x for x in df["Lehrer.übereinstimmend_clean"] if x == 1.0]
wert2 = len(wert)

diff = wert2/wert1

print(diff)
# Ergebnis: 0.508336168764886

0.508336168764886


In [4]:
df0 = df.drop(["Fragebogen-Nr._Maurer", "lfd_Maurer", "GID", "Ort", "LONG", "LAT", "Bearbeiten/in_DWA", "Digi_Index", "Name.d..Lehrers_DWA", "Geburtsort.d..Lehrers_DWA", "Lehrer.übereinstimmend", "Korrektur", "übereinst..Lehrer.ist.Einsender", "übereinst..Lehrer.ist.Beantworter", "überprüft.von", "Dubletten.DWA"], axis = 1)

## 2. Ansatz

Der Datensatz wird mittels der Orts-ID (GID) zusammengefasst und für jeden Ort ein Mittelwert gebildet.
Bei dieser Berechnung kommt man auch knapp 60% Übereinstimmung. Dieser Wert gibt an, dass an durchschnittlich 60% der Orte die gleiche Lehrkraft in die Beantwortung oder Einsendung der Fragebögen involviert war.

In [5]:
df1 = df0.groupby("coord").mean()
df1 = df1.fillna(0)

wert1 = len(df1["Lehrer.übereinstimmend_clean"])
wert = [x for x in df1["Lehrer.übereinstimmend_clean"] if x != 0.0]
wert2 = sum(wert)

diff = wert2/wert1
print(diff)

# Ergebnis: 0.5965142695746147

0.5965142695746147


## 3. Ansatz

Hier wird die selbe Idee wie bei Ansatz 2 verfolgt. Allerdings wird statt des Mittelwertes der Maximalwert für jeden Ort angegeben. Das bedeutet, dass man einen Ort, an dem mehrere Fragebögen und mehrere Lehrer beteiligt waren, als übereinstimmend wertet, wenn mindestens eine dieser Personen bei beiden Befragungen beteiligt war.
Mit dieser Methode kann davon ausgegangen werden, dass an ca. 63 % der Orte die gleiche Lehrkraft in die Beantwortung oder Einsendung der Fragebögen involviert war.

In [6]:
df3 = df0.groupby("coord").max()
df3 = df3.fillna(0)

wert1 = len(df3["Lehrer.übereinstimmend_clean"])
wert = [x for x in df3["Lehrer.übereinstimmend_clean"] if x != 0.0]
wert2 = sum(wert)

diff = wert2/wert1
print(diff)

# Ergebnis: 0.6302681992337165

0.6302681992337165


## 4. Ansatz

Im Folgenden wird die Lehrerdatei noch ergänzt um die Fragebögen, die in der Lehrerdatei nicht vorkommen. Disen Orte wird eine Übereinstimmung von "0.0" (also keine Übereinstimmung) zugewiesen.

Danach werden sowohl die unbereinigte Berechnung, als auch die Mittelwert- und Maximummethode nochmals durchgeführt. Die Ergebnisse sind entsprechend etwa 47%, 53% und 56% Übereinstimmung.

In [7]:
for fragebogen in df["Fragebogen-Nr._Maurer"]:
    for element in df2["Fragebogen Nr."]:
        if fragebogen == element:
            index = df2.loc[df2["Fragebogen Nr."] == element]["Unnamed: 0"]
            #print(index)
            df2 = df2.drop(index, axis = 0)
            #print(len(df2))

for digi in df["Digi_Index"]:
    for element in df2["Fragebogen Nr."]:
        if digi == element:
            index = df2.loc[df2["Fragebogen Nr."] == element]["Unnamed: 0"]
            #print(index)
            df2 = df2.drop(index, axis = 0)
            #print(len(df2))
    

In [8]:
# 279 Fragebögen konnten zur Lehrerdatei hinzugefügt werden
print(len(df2))

279


In [9]:
# die zwei DataFrames werden über die Fragebogen-Nummer zusammengeführt 
df2.columns = df2.columns.str.replace("Fragebogen Nr.", "Fragebogen-Nr._Maurer")
df2 = df2.fillna(0)
a = pd.concat([df, df2], axis = 0)
a = a.fillna(0)

In [10]:
wert1 = len(a["Lehrer.übereinstimmend_clean"])
wert = [x for x in a["Lehrer.übereinstimmend_clean"] if x == 1.0]
wert2 = len(wert)
diff = wert2/wert1

print(diff)

# Ergebnis: 0.4642635177128651

0.4642635177128651


In [11]:
b = pd.DataFrame()
b["coord"] = a["coord"]
b["Lehrer.übereinstimmend_clean"] = a["Lehrer.übereinstimmend_clean"]

In [12]:
a1 = b.groupby("coord").mean()

wert1 = len(a1["Lehrer.übereinstimmend_clean"])
wert = [x for x in a1["Lehrer.übereinstimmend_clean"] if x != 0.0]
wert2 = sum(wert)

diff = wert2/wert1
print(diff)

# Ergebnis: 0.5286380293205185

0.5286380293205185


In [13]:
a2 = b.groupby("coord").max()

wert1 = len(a2["Lehrer.übereinstimmend_clean"])
wert = [x for x in a2["Lehrer.übereinstimmend_clean"] if x != 0.0]
wert2 = sum(wert)

diff = wert2/wert1
print(diff)

# Ergebnis: 0.5590484282073067

0.5590484282073067
